In [1]:
import os
import util
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Input, Rescaling, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score

In [2]:
base_dir = 'artifacts'
data = os.path.join(base_dir, 'split_3d_data.pkl')
batch_size = 32

# Load Data

In [3]:
X_train, X_val, X_test, y_train, y_val, y_test = util.load_split_3d_data(data)

In [4]:
X_train.shape, X_test.shape, X_val.shape

((280, 16, 224, 224, 3), (60, 16, 224, 224, 3), (60, 16, 224, 224, 3))

In [5]:
num_frames, img_size = X_train.shape[1], X_train.shape[2:4]
print(num_frames, img_size)

16 (224, 224)


In [6]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

## Factorizing Target

In [7]:
y_val

array(['real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'real', 'real',
       'real', 'real', 'real', 'real', 'real', 'real', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake',
       'fake', 'fake', 'fake', 'fake'], dtype='<U4')

In [8]:
pd.factorize(y_val)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array(['real', 'fake'], dtype='<U4'))

In [9]:
labels = pd.factorize(y_val)[1]
print(labels)

['real' 'fake']


In [11]:
y_train, y_val, y_test = pd.factorize(y_train)[0], pd.factorize(y_val)[0], pd.factorize(y_test)[0]

In [12]:
y_train.shape, y_test.shape, y_val.shape

((280,), (60,), (60,))

## Converting 3D to 2D Data

In [13]:
num_train = X_train.shape[0]
num_val = X_val.shape[0]
num_test = X_test.shape[0]

In [14]:
train_frames, val_frames, test_frames = util.convert_3d_to_2d(split=3,
                                                         train=(X_train, y_train),
                                                         val=(X_val,y_val),
                                                         test=(X_test, y_test))
X_train_frames, y_train_frames = train_frames
X_val_frames, y_val_frames = val_frames
X_test_frames, y_test_frames = test_frames

In [15]:
X_train_frames.shape, X_val_frames.shape, X_test_frames.shape

((4480, 224, 224, 3), (960, 224, 224, 3), (960, 224, 224, 3))

In [16]:
y_train_frames.shape, y_val_frames.shape, y_test_frames.shape

((4480,), (960,), (960,))

# Simple CNN Model

In [17]:
model = Sequential()
model.add(Input(shape=img_size+(3,)))
model.add(Rescaling(1./255))
model.add(Conv2D(32, 3, strides=1, padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Conv2D(64, 3, strides=1, padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Conv2D(128, 3, strides=1, padding='valid', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=2, padding='valid'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)                │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(learning_rate=0.01), 
              metrics=['accuracy'])

In [19]:
estop = EarlyStopping(monitor='val_loss', mode='min',
                      min_delta=1e-5, patience=10,
                      restore_best_weights=True,
                      verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5,
                              min_delta=1e-3, min_lr=1e-5, verbose=1)

In [20]:
model.fit(X_train_frames, y_train_frames, 
          validation_data=(X_val_frames, y_val_frames),
          epochs=500, batch_size=batch_size,
          callbacks=[estop, reduce_lr],
          verbose=1)

Epoch 1/500
140/140 ━━━━━━━━━━━━━━━━━━━━ 45s 309ms/step - accuracy: 0.5009 - loss: 1.6654 - val_accuracy: 0.5000 - val_loss: 0.6932 - learning_rate: 0.0100
Epoch 2/500
140/140 ━━━━━━━━━━━━━━━━━━━━ 43s 304ms/step - accuracy: 0.4964 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6937 - learning_rate: 0.0100
Epoch 3/500
140/140 ━━━━━━━━━━━━━━━━━━━━ 43s 306ms/step - accuracy: 0.4998 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6932 - learning_rate: 0.0100
Epoch 4/500
140/140 ━━━━━━━━━━━━━━━━━━━━ 43s 307ms/step - accuracy: 0.4971 - loss: 0.6933 - val_accuracy: 0.5000 - val_loss: 0.6935 - learning_rate: 0.0100
Epoch 5/500
140/140 ━━━━━━━━━━━━━━━━━━━━ 43s 306ms/step - accuracy: 0.4866 - loss: 0.6935 - val_accuracy: 0.5000 - val_loss: 0.6932 - learning_rate: 0.0100
Epoch 6/500
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.4916 - loss: 0.6935 
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.004999999888241291.
140/140 ━━━━━━━━━━━━━━━━━━━━ 43s 307ms/step - accuracy:

In [21]:
test_loss, test_accuracy = model.evaluate(X_test_frames, y_test_frames, verbose=0)
print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)

Test loss: 0.6931546330451965
Test accuracy: 0.5


**Simple CNN model gave a (frame-level) test accuracy of 50%.**

In [22]:
model.save('artifacts/baseline_cnn.keras')

In [30]:
frame_pred_probs = model.predict(X_test_frames, verbose=0).reshape(num_test, num_frames, -1)
frame_pred_probs.shape

(60, 16, 1)

In [31]:
print(frame_pred_probs[1])

[[0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]
 [0.4980377]]


In [32]:
frame_pred_labels = (frame_pred_probs>0.5).astype(int)
print(frame_pred_labels.shape)
print(frame_pred_labels[1])

(60, 16, 1)
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


In [33]:
video_pred_labels = (np.sum(frame_pred_labels, axis=1) > (num_frames/2)).astype(int)
print(video_pred_labels.shape)

(60, 1)


In [34]:
video_test_accuracy = accuracy_score(y_test, video_pred_labels)
print("Video-level Test accuracy:", video_test_accuracy)

Video-level Test accuracy: 0.5


**With majority voting, frame predictions are combined to obtain video predictions. The video-level test accuracy achieved is 50%.\
This serves as the baseline model which appears to be working no better than random guessing.**